In [338]:
import pandas  
import numpy
from pybaseball import schedule_and_record
from pybaseball import batting_stats
from pybaseball import bwar_bat
from pybaseball import bwar_pitch
from pybaseball import pitching_stats
from pybaseball import batting_stats
from datetime import timedelta, date
import time

In [ ]:
# game results and schedule
teams_list = ['CHC','CIN','STL','PIT','MIL','LAD','ARI','COL','SDP','SFG','MIA','PHI','NYM','ATL','WSN',
             'CHW','CLE','KCR','DET','MIN','HOU','OAK','SEA','LAA','TEX','NYY','TBR','BOS','TOR','BAL']

def results(x):
    schedule = schedule_and_record(2020, x)
    schedule['W/L'].replace('W-wo','W', inplace=True)
    schedule['W/L'].replace('L-wo','L', inplace=True)
    return(schedule)
schedule_results = pandas.DataFrame()

for team in teams_list:
    sched = results(team)
    schedule_results = pandas.concat([schedule_results,sched],ignore_index=True)
schedule_results

In [ ]:
# export hitter war
bwar_data = bwar_bat()
bwar_data = bwar_data[(bwar_data['year_ID'] == 2019)]
bwar_data = bwar_data[['name_common','mlb_ID','pitcher','G','PA','salary','WAR']]
bwar_data.to_csv('batting_war.csv')
# export pitcher war
pwar_data = bwar_pitch()
pwar_data = pwar_data[(pwar_data['year_ID'] == 2019)]
pwar_data = pwar_data[['name_common','G','GS','RA','xRA','BIP_perc','salary','WAR']]
pwar_data.to_csv('pitching_war.csv')
# export pitching stats
pitch = pitching_stats(start_season=2019)
pitch.to_csv('pitching_stats.csv')
# export hitter stats
data = batting_stats(start_season=2019, end_season=2019)
data.to_csv('batting_stats.csv')

In [341]:
%%time

# returns pitch by pitch data for all games between specified dates
from pybaseball import statcast
from datetime import timedelta, date

statcast_df = pandas.DataFrame()
start_date = date(2020,10,20)
end_date = date(2020,10,31)
t = 0
dates_of_pull = pandas.date_range(start_date,end_date-timedelta(days=1),freq='d')

for i in range(len(dates_of_pull)):
    exact_date = dates_of_pull[t].strftime('%Y-%m-%d')
    statcast_pull = statcast(start_dt=exact_date,end_dt=exact_date)
    statcast_pull = statcast_pull.drop(columns=['des'])
    statcast_df = pandas.concat([statcast_df,statcast_pull],axis=0)
    t += 1
    time.sleep(1)
    
# statcast_df.to_csv('statcast.csv')

Wall time: 17.7 s


In [335]:
#find missing players information
from bs4 import BeautifulSoup
from requests_html import HTMLSession
session = HTMLSession()
from pandas.io.json import json_normalize
import json
import pandas
import numpy
from datetime import date

names =[]
positions = []
batthrow = []
htwt = []
born = []
ids =[]

#input mlb player ids to get player bio info
player_ids = ['657656','669257']
#

headers = {
           'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
           'accept-encoding': 'gzip, deflate, br',
           'accept-language': 'en-US,en;q=0.8',
           'upgrade-insecure-requests': '1',
           'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'
        }

for i in player_ids:
    url = "https://www.mlb.com/player/"+str(i)
    content = session.get(url, headers=headers)
    soup = BeautifulSoup(content.text, "html.parser")
    ids.append(str(i))
    try:
        names.append(soup.find("main").find("h1").find_all("span")[0].text)
    except:
        names.append('none')
    try:
        positions.append(soup.find("main").find("ul").find_all("li")[0].text)
    except:
        positions.append('none')
    try:
        batthrow.append(soup.find("main").find("ul").find_all("li")[1].text)
    except:
        batthrow.append('none')
    try:
        htwt.append(soup.find("main").find("ul").find_all("li")[2].text)
    except:
        htwt.append('none')
    try:
        born.append(soup.find(lambda tag: tag.name == 'span' and tag.get_text(strip=True) == 'Born:').find_next_sibling(text=True))
    except:
        born.append('none')
player_data = pandas.DataFrame(dict(player_id=ids,name=names,position=positions,bat_throw=batthrow,height_weight=htwt,born_place=born))
player_data = player_data.set_index('player_id')
# player_data.to_csv('mlb_missing_ids.csv')